# Aircraft Object Detection Notebook

### Import Dependencies

In [80]:
!pip install wget #required to download protoc compiler
!pip install scikit-learn #required for train-test-split.py
!pip install opencv-python
!pip install gin-config  

  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl (38.1 MB)


In [81]:
import os
import wget
import cv2

### 1. Files

In [28]:
#configure these based on model choice (Tensorflow 2 object detection zoo)
CUSTOM_MODEL_NAME = 'resnet_inception' 
PRETRAINED_MODEL_NAME ='faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL='http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz'

In [17]:
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
TRAIN_TEST_SCRIPT_NAME = 'train-test-split.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

### 2. Setup Folders and Files


In [18]:
filepaths = {
    'COLLECTIMAGE': os.path.join('Tensorflow','workspace','images','collectedimages'),
    'IMAGE_PATH': os.path.join('Tensorflow','workspace','images'),
    'TRAIN_PATH': os.path.join('Tensorflow','workspace','images','train'),
    'TEST_PATH': os.path.join('Tensorflow','workspace','images','train'),
    'SCRIPTS': os.path.join('Tensorflow','scripts'),
    'FROMAPIMODEL': os.path.join('Tensorflow','models'),
    'OURMODEL': os.path.join('Tensorflow','workspace','models'),
    'PRETRAINMODEL': os.path.join('Tensorflow','workspace','pre-trained-models'),
    'TFJS_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH': os.path.join('Tensorflow','protoc'),
    'ANNOTATION': os.path.join('Tensorflow','workspace','annotations'),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow','workspace','models',CUSTOM_MODEL_NAME)
}
    
files = {
    'PIPELINE_CONFIG':os.path.join(filepaths['OURMODEL'], CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(filepaths['SCRIPTS'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(filepaths['ANNOTATION'], LABEL_MAP_NAME),
    'TRAIN_TEST_SPLIT': os.path.join(filepaths['SCRIPTS'], TF_RECORD_SCRIPT_NAME)
    
}


In [20]:
for path in filepaths.values():
    if not os.path.exists(path):
        !mkdir {path}

In [21]:
#Populate label_map.pbtxt with labels (here we have choosen 'Airplane' and 'Non-Airplane'

labels = [{'name':'Airplane', 'id':1}, {'name':'Non-Airplane', 'id':2}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [22]:
if not os.path.exists(os.path.join(filepaths['FROMAPIMODEL'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {filepaths['FROMAPIMODEL']}

In [11]:
url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
wget.download(url)
!move protoc-3.15.6-win64.zip {filepaths['PROTOC_PATH']}
!cd {filepaths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(filepaths['PROTOC_PATH'], 'bin'))   
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd Tensorflow/models/research/slim && pip install -e . 

100% [..........................................................................] 1468733 / 1468733        1 file(s) moved.
        1 file(s) copied.

C:\Users\james\Aircraft Project\odn\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
C:\Users\james\Aircraft Project\odn\lib\site-packages\setuptools\command\easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
zip_safe flag not set; analyzing archive contents...
object_detection.core.__pycache__.densepose_ops.cpython-310: module references __file__
object_detection.core.__pycache__.preprocessor.cpython-310: module MAY be using inspect.stack
object_detection.utils.__pycache__.autoaugment_utils.cpython-310: module MAY be using inspect.stack
No local packages or working download links found for tensorflow-text~=2.14.0
error: Could not find suitable distribution for Requirement.parse('tensorflow-text~=2.14.0')



running build
running build_py
copying object_detection\protos\anchor_generator_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\argmax_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\bipartite_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_predictor_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\calibration_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\center_net_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\eval_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\flexible_grid_anchor_generator_pb2.py ->

### 2.5 Run verification script


##### run these two cells, installing the libraries/modules below as necessary, repeatedly until running the "!python {VERIFICATION_SCRIPT}" line outputs a (maybe red) wall of text with "OK" printed at the very bottom

In [23]:
#install modules and bug-fix until this runs ok and prints "OK" at bottom of text-wall (will be red but will work)
VERIFICATION_SCRIPT = os.path.join(filepaths['FROMAPIMODEL'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

Running tests under Python 3.10.9: C:\Users\james\Aircraft Project\odn\Scripts\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2023-10-26 20:43:12.422176: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE SSE2 SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
W1026 20:43:12.532587 24540 batch_normalization.py:1531] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
C:\Users\james\Aircraft Project\odn\lib\site-packages\object_detection-0.1-py3.10.egg\object_detection\builders\model_builder.py:1112: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(('Buildin

In [ ]:
#run each of these lines (remove hashtag) to install modules that may be causing the VERIFICATION_SCRIPT to fail

#!pip install tensorflow
#!pip install protobuf
#!pip install matplotlib==3.2
#!pip install cycler 
#!pip install kiwisolver
#!pip install contourpy

### 3. Move SoTA model from API into correct path

In [24]:
import object_detection  # now we can import this module

In [25]:
wget.download(PRETRAINED_MODEL_URL)
!move {PRETRAINED_MODEL_NAME+'.tar.gz'} {filepaths['PRETRAINMODEL']}
!cd {filepaths['PRETRAINMODEL']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

100% [......................................................................] 444343592 / 444343592

The system cannot find the file specified.
tar: Error opening archive: Failed to open 'faster_r-cnn_resnet_v2_640x640_coco17_tpu-8.tar.gz'


### 3.5 Perform train/test split

In [ ]:
!mv train-test-split.py filepaths['SCRIPTS']

In [70]:
!python {files['TRAIN_TEST_SPLIT']}

Traceback (most recent call last):
  File "C:\Users\james\Aircraft Project\Tensorflow\scripts\train-test-split.py", line 26, in <module>
    train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=random_seed)
  File "C:\Users\james\Aircraft Project\odn\lib\site-packages\sklearn\utils\_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\james\Aircraft Project\odn\lib\site-packages\sklearn\model_selection\_split.py", line 2649, in train_test_split
    n_train, n_test = _validate_shuffle_split(
  File "C:\Users\james\Aircraft Project\odn\lib\site-packages\sklearn\model_selection\_split.py", line 2305, in _validate_shuffle_split
    raise ValueError(
ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


### 4. Convert xml files (labels) to TF records

In [72]:
!cd {filepaths['SCRIPTS']} && mkdir gitTFRecord

A subdirectory or file gitTFRecord already exists.


In [73]:
#The python script (generate_tfrecord.py) from this github repo converts xml files into TF records, its file path is files['TF_RECORD_SCRIPT']
if TF_RECORD_SCRIPT_NAME not in os.listdir(os.path.join(filepaths['SCRIPTS'],'gitTFRecord')):
    !git clone  https://github.com/nicknochnack/GenerateTFRecord {(os.path.join(filepaths['SCRIPTS'],'gitTFRecord'))}

In [74]:
!copy {os.path.join(filepaths['SCRIPTS'],'gitTFRecord')} {filepaths['SCRIPTS']}

Tensorflow\scripts\gitTFRecord\generate_tfrecord.py
        1 file(s) copied.


In [52]:
#this code uses the generate_tfrecord.py script to convert xml to TFRecord for the train set and then the test set
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(filepaths['TRAIN_PATH'])} -l {files['LABELMAP']} -o {os.path.join(filepaths['ANNOTATION'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(filepaths['TEST_PATH'])} -l {files['LABELMAP']} -o {os.path.join(filepaths['ANNOTATION'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow\workspace\annotations\train.record
Successfully created the TFRecord file: Tensorflow\workspace\annotations\test.record


In [51]:
#if the above cell doesn't work, uncheck the next line
#!pip install pytz

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas 2.1.1 requires tzdata>=2022.1, which is not installed.
apache-beam 2.51.0 requires cloudpickle~=2.2.1, which is not installed.
apache-beam 2.51.0 requires crcmod<2.0,>=1.7, which is not installed.
apache-beam 2.51.0 requires dill<0.3.2,>=0.3.1.1, which is not installed.
apache-beam 2.51.0 requires fastavro<2,>=0.23.6, which is not installed.
apache-beam 2.51.0 requires fasteners<1.0,>=0.3, which is not installed.
apache-beam 2.51.0 requires hdfs<3.0.0,>=2.1.0, which is not installed.
apache-beam 2.51.0 requires httplib2<0.23.0,>=0.8, which is not installed.
apache-beam 2.51.0 requires js2py<1,>=0.74, which is not installed.
apache-beam 2.51.0 requires objsize<0.7.0,>=0.6.1, which is not installed.
apache-beam 2.51.0 requires orjson<4,>=3.9.7, which is not installed.
apache-beam 2.51.0 requires proto-plus<2,


  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)


### 5. Moving Model Config file

In [53]:
#move config file from the standard pretrained model ('template') file to the file for our specific version of the model
!copy {os.path.join(filepaths['PRETRAINMODEL'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(filepaths['CHECKPOINT_PATH'])}

        1 file(s) copied.


### 6. Updating configuration - Transfer Learning

In [54]:
#these should now import fine
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [55]:
#modify configuration
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [56]:
config

{'model': faster_rcnn {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "faster_rcnn_inception_resnet_v2_keras"
     batch_norm_trainable: true
   }
   first_stage_anchor_generator {
     grid_anchor_generator {
       height_stride: 16
       width_stride: 16
       scales: 0.25
       scales: 0.5
       scales: 1
       scales: 2
       aspect_ratios: 0.5
       aspect_ratios: 1
       aspect_ratios: 2
     }
   }
   first_stage_box_predictor_conv_hyperparams {
     op: CONV
     regularizer {
       l2_regularizer {
         weight: 0
       }
     }
     initializer {
       truncated_normal_initializer {
         stddev: 0.01
       }
     }
   }
   first_stage_nms_score_threshold: 0
   first_stage_nms_iou_threshold: 0.7
   first_stage_max_proposals: 300
   first_stage_localization_loss_weight: 2
   first_stage_objectness_loss_weight: 1
   initial_crop_size: 17
   maxpool_kernel_siz

In [57]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [61]:
pipeline_config.model.faster_rcnn.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(filepaths['PRETRAINMODEL'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(filepaths['ANNOTATION'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(filepaths['ANNOTATION'], 'test.record')]

In [3]:
# Adjust these params (generalise)
# Aircraft Non-Aircraft

### 7. Training of Model

In [63]:
TRAINING_SCRIPT = os.path.join(filepaths['FROMAPIMODEL'], 'research', 'object_detection', 'model_main_tf2.py')

In [64]:
#the number of training steps can be adjusted appropriately
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, filepaths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [65]:
# run this training command in command prompt so you can track its progress properly (copy paste the output of this)
print(command)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\resnet_inception --pipeline_config_path=Tensorflow\workspace\models\resnet_inception\pipeline.config --num_train_steps=2000


In [82]:
#or remove the hashtag and run the training command in notebook
!{command}

Traceback (most recent call last):
  File "C:\Users\james\Aircraft Project\Tensorflow\models\research\object_detection\model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
  File "C:\Users\james\Aircraft Project\odn\lib\site-packages\object_detection-0.1-py3.10.egg\object_detection\model_lib_v2.py", line 31, in <module>
    from object_detection import model_lib
  File "C:\Users\james\Aircraft Project\odn\lib\site-packages\object_detection-0.1-py3.10.egg\object_detection\model_lib.py", line 35, in <module>
    from object_detection.builders import optimizer_builder
  File "C:\Users\james\Aircraft Project\odn\lib\site-packages\object_detection-0.1-py3.10.egg\object_detection\builders\optimizer_builder.py", line 25, in <module>
    from official.modeling.optimization import ema_optimizer
  File "C:\Users\james\Aircraft Project\odn\lib\site-packages\tf_models_official-2.14.2-py3.10.egg\official\modeling\optimization\__init__.py", line 23, in <module>
    

### 8. Evaluate the Model

In [ ]:
command_eval = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, filepaths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], filepaths['CHECKPOINT_PATH'])

In [ ]:
#run evaluation command in command prompt to track properly (copy paste the output)
print(command_eval)

In [75]:
#or remove the hashtag and run in notebook
#!{command_eval}